**Prompting** is the process of providing a partial, usually text, input to a model. As we discussed in the last chapter, models will then use their parameterized data transformations to find a probable completion or output that matches the prompt.

**Prompt and AI Engineering** is the emerging developer task of designing and optimizing prompts (and associated workflows/ infra) for AI models to achieve specific goals or outcomes. It involves creating high-quality inputs that can elicit accurate and relevant responses from AI models. The next several examples will help get you up to speed on common prompt engineering strategies.

```
               +-------------------+
               |                   |
               |                   |  Completion
Prompt         |       Large       |  Generated text
--------------->     Language      +------------->
               |       Model       |
               |       (LLM)       |
               |                   |
               +-------------------+
```

# Dependencies and imports

In [1]:
! pip install predictionguard

In [2]:
import os

from predictionguard import PredictionGuard
from getpass import getpass

In [3]:
pg_access_token = getpass('Enter your Prediction Guard access api key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

Enter your Prediction Guard access api key: ··········


In [4]:
client = PredictionGuard()

# Autocomplete

Because LLMs are configured/ trained to perform the task of text completion, the most basic kind of prompt that you might provide is an **autocomplete** prompt. Regardless of prompt structure, the model function will compute the probabilities of words, tokens, or characters that might follow in the sequence of words, tokens, or characters that you provided in the prompt.

Depending on the desired outcome, the prompt may be a single sentence, a paragraph, or even an partial story. Additionally, the prompt may be open-ended, providing only a general topic or theme, or it may be more specific, outlining a particular scenario or plot.

In [14]:
result = client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": "Daniel Whitenack, a long forgotten wizard from the Lord of the Rings, entered into Mordor to"}]
)

print(result['choices'][0]['message']['content'])

retrieve the One Ring and destroy it in the fires of Mount Doom. However, he was not mentioned in J.R.R. Tolkien's novels or any officially licensed adaptations of the Lord of the Rings.

If Daniel Whitenack were to enter Mordor, he would face immense challenges and dangers, including:

1. The hostile landscape: Mordor is a harsh, barren land, with little food, water, or shelter for travelers. Its extreme conditions would make it difficult for anyone to survive


In [15]:
result = client.chat.completions.create(
    model="Neural-Chat-7B",
    messages=[{"role": "user", "content": "Today I inspected the engine mounting equipment. I found a problem in one of the brackets so"}]
)

print(result['choices'][0]['message']['content'])

you need to report this issue. Please document the faulty bracket carefully, noting its location, any visible damage, or worn-out components. Next, communicate the problem to the appropriate people, such as your supervisor, the head of the maintenance department, or the individual responsible for engine mounting. Make sure to include all the relevant details you've gathered from your inspection to help expedite the solution process. If necessary, make suggestions for the replacement or repair of the damaged part.


In [18]:
result = client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": """Complete this SQL statement:

CREATE TABLE llm_queries(id SERIAL PRIMARY KEY, name TEXT NOT NULL, value REAL);
INSERT INTO llm_queries('Daniel Whitenack', 'autocomplete')
SELECT"""}]
)

print(result['choices'][0]['message']['content'])

To complete the SQL statement, you can insert the values directly into the table using the VALUES clause. Here's the updated SQL statement:

CREATE TABLE llm_queries(id SERIAL PRIMARY KEY, name TEXT NOT NULL, value TEXT);
INSERT INTO llm_queries(id, name, value) VALUES 
(DEFAULT, 'Daniel Whitenack', 'autocomplete');

Explanation:
- The first part of the statement creates a table named "llm_queries" with three columns: "id" (auto-incrementing integer


# Zero shot, Instruction prompts

Autocomplete is a great place to start, but it is only that: a place to start. Throughout this workshop we will be putting on our prompt engineering hats to do some impressive things with generative AI. As we continue along that path, there is a general prompt structure that will pop up over and over again:

```
 Prompt:
+------------------------------------------------------------+
|                                                            |
|  +-------------------------------------------------------+ |
|  | ----------------------------------------------------- | | Task Descrip./
|  | ---------------------------------------               | | Instructions
|  +-------------------------------------------------------+ |
|                                                            |
|  +-------------------------------------------------------+ | Current Input/
|  | -------------                                         | | Context
|  +-------------------------------------------------------+ |
|                                                            |
|  +----------------------------------------+                | Output
|  | --------------------------             |                | Indicator
|  +----------------------------------------+                |
|                                                            |
+------------------------------------------------------------+
```

One of the easiest ways to leverage the above prompt structure is to describe a task (e.g., sentiment analysis), provide a single piece of data as context, and then provide a single output indicator. This is called a **zero shot prompt**.

Note, sometimes you can separate out the task description or instructions into the system prompt, but not always. It likely depends on how boilerplate this is between calls to the LLM.

## Sentiment analysis

In [19]:
result = client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": """Respond with a sentiment label for the text included in the below user input. Use the label NEU for neutral sentiment, NEG for negative sentiment, and POS for positive sentiment. Respond only with one of these labels and no other text.

Input:
This tutorial is spectacular. I love it! So wonderful.

Response: """}]
)

print(result['choices'][0]['message']['content'])

POS


## Question Answering

In [20]:
prompt = """Read the context in the below input and respond with an answer to the question. If the question cannot be answered based on the context alone or the context does not explicitly say the answer to the question, write "Sorry I had trouble answering this question, based on the information I found."

Context: Domino's gift cards are great for any person and any occasion. There are a number of different options to choose from. Each comes with a personalized card carrier and is delivered via US Mail.

Question: How are gift cards delivered?

Response: """

In [21]:
client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": prompt}]
)['choices'][0]['message']['content']

'The gift cards are delivered via US Mail.'

# Few Shot prompts

When your task is slightly more complicated or requires a few more leaps in reasoning to generate an appropriate response, you can turn to **few shot** prompting (aka **in context learning**). In few shot prompting, a small number of gold standard demonstrations are integrated into the prompt. These demonstrations serve as example (context, output) pairs for the model, which serve to tune the probable output on-the-fly to what we ideally want in the output.

Although not always necessary (as seen above), few shot prompting generally produces better results than single shot prompting in terms of consistency and similarity to your ideal outputs. This does come at a cost for some models that might charge based on the number of characters or words that are input to the model API.

## Sentiment

In [22]:
prompt = """Classify the sentiment of the text. Here are some examples:

Text: That pilot is adorable.
Sentiment: {"label": "POS"}

Text: This was an awful seat.
Sentiment: {"label": "NEG"}

Text: The flight is boring.
Sentiment: """

In [23]:
client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": prompt}]
)['choices'][0]['message']['content']

'{"label": "NEG"}'

## Chat, Hinglish conversation

In [24]:
prompt = """The following is a conversation with an AI assistant in Hinglish. The assistant is helpful, creative, clever, and very friendly. The AI assistant responds in Hinglish.

Human: Hello
AI: Hello!

Human: Movie ke baare mein tumhara initial opinion kya hai? Zyada depth mein gaye bina? Tumne like kiya ki nahin?
AI: Overall, mujhe movie achchi lagi.  Usme kuh high-profile actors hain,  Tum kya kahte ho?

Human: Tumhe lagta hai ki ek achche movie mein high-profile actors hone chahiye? Ya phir, ek movie ko achcha kahne ke liye high-profile actors hone chahiye?
AI: Zaroori nahin, par help zaroor milta hai.  Ek actor jab tak apna kaam theek se nahin karta, tab tak high-profile nahin ban sakta. Tumhe kya lagta hai?

Human: muje lgta hai ki yeh kewal personal pasand ke bare mai hai meri pasandida american comedy films Will Ferrol or Seth Rogan ke sath kuch bhi samil slapstick films hai muje yakin hai ki ek film thi jisme Lindsay Lohan ko bhot bada bna diya tha bhale hi usne apne surati saalo mai movies mai acting ki thi
AI: """

In [26]:
output = client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": prompt}]
)['choices'][0]['message']['content']

print(output)

Abhi ka Matlab yeh hai ki actor ka talent important hai, but yeh bataate hai ki meri ek personal pasand hai American comedy films, jisme Will Ferrell aur Seth Rogen common actors hain, aur kuch slapstick films bhi. Mujhe yeh aasaani se lagey kyonki in actors aur comedy films ke baare mein mujhe bahut knowledge hai. But movie ki quality ka naya sacha matlab hai high-profile actors


In [30]:
prompt = """Translate the following Hinglish input to English.

Hinglish input:
{hinglish}

English translation:
""".format(hinglish=output)

client.chat.completions.create(
    model="Hermes-3-Llama-3.1-8B",
    messages=[{"role": "user", "content": prompt}]
)['choices'][0]['message']['content']

'The meaning of "right now" is that an actor\'s talent is important, but it also shows that I have a personal preference for American comedy films, in which Will Ferrell and Seth Rogen are common actors, and some slapstick films as well. It becomes easy for me because I have a lot of knowledge about these actors and comedy films. But the real meaning of movie quality is high-profile actors.'